In [48]:
import cv2 
import matplotlib.pylab as plt
import numpy as np
from scipy import fftpack

In [49]:
# This is the convolution function from Assignment 1
def convolution_image(input_image, conv_kernel):  
    # Kernel dimension
    x_kern = conv_kernel.shape[0]
    y_kern = conv_kernel.shape[1]
    
    # Kernel is flipped both horizontally and vertically
    kernel_lr = np.fliplr(conv_kernel)
    conv_kernel = np.flipud(kernel_lr)
    
    # Add zero padding to the input image
    input_image_pad = np.zeros((input_image.shape[0] + 2, input_image.shape[1] + 2))
    input_image_pad[1:-1, 1:-1] = input_image
    
    # Initialize the output image of convolution
    conv_output = np.zeros_like(input_image)

    # For every pixel of the image, do multiplication with the kernel
    for m in range(input_image.shape[1]):
        for n in range(input_image.shape[0]):
            conv_output[n, m]=(conv_kernel * input_image_pad[n: n + y_kern, m: m + x_kern]).sum()

    return conv_output


In [50]:
### Q1: Filter 
kernel_posneg = np.array([[0,-1/4,0], [-1/4,0,-1/4], [0,-1/4,0]])

In [51]:
### Q2: Filter 
kernel_xgrad = np.array([[0,-1,1]])

In [61]:
### Q3: Transformations in Frequency Domain
## Read the image
img = cv2.imread("sample1.jpg",0)
cv2.imwrite('sample1_gray.jpg',img)  

True

In [62]:
### Q3.1 and Q3.2 Defining horizontal and vertical edge detection filter
horizontal_edge_fl = np.array([[+1,+2,+1], [0,0,0], [-1,-2,-1]])
vertical_edge_fl = np.array([[-1,0,+1], [-2,0,+2], [-1,0,+1]])

In [63]:
### How horizontal and vertical edge filters work on the image (Convolving them with the image in spatial domain)
img_hor_edge = convolution_image(img,horizontal_edge_fl)
cv2.imwrite('sample1_conv_hor_edge.jpg',img_hor_edge) 

img_ver_edge = convolution_image(img,vertical_edge_fl)
cv2.imwrite('sample1_conv_ver_edge.jpg',img_ver_edge) 

True

In [64]:
## Q3.3 Frequency domain transformation of vertical and horizontal filters
f_horizontal = fftpack.fft2(horizontal_edge_fl)
f_horizontal_shift = fftpack.fftshift(f_horizontal)
f_horizontal_fdomain = 20 * np.log(np.abs(f_horizontal_shift))

f_vertical = fftpack.fft2(vertical_edge_fl)
f_vertical_shift = fftpack.fftshift(f_vertical)
f_vertical_fdomain = 20 * np.log(np.abs(f_vertical_shift))

cv2.imwrite('hor_edge_filter_fft.jpg',f_horizontal_fdomain) 
cv2.imwrite('ver_edge_filter_fft.jpg',f_vertical_fdomain) 


<ipython-input-64-6f4a14d35207>:4: RuntimeWarning: divide by zero encountered in log
  f_horizontal_fdomain = 20 * np.log(np.abs(f_horizontal_shift))
<ipython-input-64-6f4a14d35207>:8: RuntimeWarning: divide by zero encountered in log
  f_vertical_fdomain = 20 * np.log(np.abs(f_vertical_shift))


True

In [66]:
## Q3.4 Frequency domain transformation of the image
f_img = fftpack.fft2(img)
f_img_shift = fftpack.fftshift(f_img)
f_img_fdomain = 20 * np.log(np.abs(f_img_shift))

cv2.imwrite('sample1_fft.jpg',f_img_fdomain) 

True

In [69]:
### Q3.5 and Q3.6 Transformation into frequency domain and then conversion into spatial domain
### This is for the horizontal edge filter
kernel_hor_edge = np.array([[+1,+2,+1], [0,0,0], [-1,-2,-1]]) ## Same as horizontal_edge_fl

sz = (img.shape[0] - kernel_hor_edge.shape[0], img.shape[1] - kernel_hor_edge.shape[1])  ## total amount of padding for the filter
kernel_hor_edge = np.pad(kernel_hor_edge, (((sz[0]+1)//2, sz[0]//2), ((sz[1]+1)//2, sz[1]//2)), 'constant') ## Add the padding to the filter
print(kernel_hor_edge.shape)

kernel_hor_edge = fftpack.ifftshift(kernel_hor_edge)
inv_fdomain_hor = np.real(fftpack.ifft2(f_img * fftpack.fft2(kernel_hor_edge)))+ np.imag(fftpack.ifft2(f_img * fftpack.fft2(kernel_hor_edge)))

cv2.imwrite('sample1_hor_fft.jpg',inv_fdomain_hor) 

(241, 321)


True

In [72]:
### Q3.5 and Q3.6 Transformation into frequency domain and then conversion into spatial domain
### This is for the vertical edge filter
kernel_ver_edge = np.array([[-1,0,+1], [-2,0,+2], [-1,0,+1]]) ## Same as vertical_edge_fl

sz = (img.shape[0] - kernel_ver_edge.shape[0], img.shape[1] - kernel_ver_edge.shape[1])  ## total amount of padding for the filter
kernel_ver_edge = np.pad(kernel_ver_edge, (((sz[0]+1)//2, sz[0]//2), ((sz[1]+1)//2, sz[1]//2)), 'constant') ## Add the padding to the filter
print(kernel_ver_edge.shape)

kernel_ver_edge = fftpack.ifftshift(kernel_ver_edge)
inv_domain_ver = np.real(fftpack.ifft2(f_img * fftpack.fft2(kernel_ver_edge)))+ np.imag(fftpack.ifft2(f_img * fftpack.fft2(kernel_ver_edge)))

cv2.imwrite('sample1_ver_fft.jpg',inv_domain_ver) 

(241, 321)


True

In [77]:
spatial_addn = np.sqrt(np.square(inv_domain_ver) + np.square(inv_fdomain_hor))
spatial_addn *= 255.0 / spatial_addn.max()
cv2.imwrite('sample1_final.jpg',spatial_addn) 

True